PLEASE NOTE: Please run this notebook OUTSIDE a Spark notebook as it should run in a plain Default Python 3.6 Free Environment

This is the last assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Just execute all cells one after the other and you are done - just note that in the last one you should update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

The purpose of this assignment is to learn how feature engineering boosts model performance. You will apply Discrete Fourier Transformation on the accelerometer sensor time series and therefore transforming the dataset from the time to the frequency domain.

After that, you’ll use a classification algorithm of your choice to create a model and submit the new predictions to the grader. Done.


In [1]:
import os
package = "systemml"
try:
    __import__package
except:
    #! pip install systemml
    os.system("pip install "+ package)

In [2]:
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))

if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

In [3]:
package = "pyspark==2.4.5"
try:
    __import__package
except:
    #! pip install systemml
    os.system("pip install "+ package)

#!pip install pyspark==2.4.5

In [4]:
!pip install https://github.com/IBM/skillsnetwork/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true

  Using cached https://github.com/IBM/skillsnetwork/blob/master/systemml-1.3.0-SNAPSHOT-python.tar.gz?raw=true (9.9 MB)
  Created wheel for systemml: filename=systemml-1.3.0-py3-none-any.whl size=9882974 sha256=257e85e50676998f245e402e8ffeefa21de32b18dc34e6f00580ce41e55080b4
  Stored in directory: c:\users\bilgi\appdata\local\pip\cache\wheels\b5\f3\65\f44f93bcc9afc8fec7730345c4697654213f4c992e4608ac56
Successfully built systemml


In [5]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.3.0 (as of 20th March'19) Please use the code block below to check if you are already on 1.3.0 or higher. 1.3 contains a necessary fix, that's we are running against the SNAPSHOT

In [7]:
if not os.path.exists('~\home\dsxuser\work\systemml'):
    ! mkdir -p ~\home\dsxuser\work\systemml
#! mkdir -p ~\home\dsxuser\work\systemml

In [8]:
from systemml import MLContext, dml
ml = MLContext(spark)
ml.setConfigProperty("sysml.localtmpdir", "mkdir /home/dsxuser/work/systemml")
print(ml.version())
    
if not ml.version() == '1.3.0-SNAPSHOT':
    raise ValueError('please upgrade to SystemML 1.3.0, or restart your Kernel (Kernel->Restart & Clear Output)')

1.3.0-SNAPSHOT


In [15]:
package = "wget"
try:
    __import__package
except:
    #! pip install wget
    os.system("pip install "+ package)
    import wget

In [17]:
url = 'https://github.com/IBM/coursera/blob/master/coursera_ml/shake.parquet?raw=true'

if os.path.exists("shake.parquet"):
  os.remove("shake.parquet")
else:
  print("The file does not exist") 

wget.download(url)

'shake.parquet'

Now it’s time to read the sensor data and create a temporary query table.

In [18]:
df=spark.read.parquet('shake.parquet')

In [19]:
df.show()

+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



In [21]:
#!pip install --user --upgrade pixiedust
package = "pixiedust"
try:
    __import__package
except:
    os.system("pip install "+ package)
    import pixiedust

Pixiedust database opened successfully


In [22]:
import pixiedust
display(df)

DataFrame[CLASS: bigint, SENSORID: string, X: double, Y: double, Z: double]

In [23]:
df.createOrReplaceTempView("df")



We’ll use Apache SystemML to implement Discrete Fourier Transformation. This way all computation continues to happen on the Apache Spark cluster for advanced scalability and performance.

As you’ve learned from the lecture, implementing Discrete Fourier Transformation in a linear algebra programming language is simple. Apache SystemML DML is such a language and as you can see the implementation is straightforward and doesn’t differ too much from the mathematical definition (Just note that the sum operator has been swapped with a vector dot product using the %*% syntax borrowed from R ):

$X_k = \sum_{n=0}^{N_1} x_n . e^{-2\pi i k n/N}$

$=\sum_{n=0}^{N-1} x_n [cos(2 \pi k n/N) - i sin(2 \pi k n/N)]$

In [24]:
dml_script = '''
PI = 3.141592654
N = nrow(signal)

n = seq(0, N-1, 1)
k = seq(0, N-1, 1)

M = (n %*% t(k))*(2*PI/N)

Xa = cos(M) %*% signal
Xb = sin(M) %*% signal

DFT = cbind(Xa, Xb)
'''

Now it’s time to create a function which takes a single row Apache Spark data frame as argument (the one containing the accelerometer measurement time series for one axis) and returns the Fourier transformation of it. In addition, we are adding an index column for later joining all axis together and renaming the columns to appropriate names. The result of this function is an Apache Spark DataFrame containing the Fourier Transformation of its input in two columns.

In [25]:
from pyspark.sql.functions import monotonically_increasing_id

def dft_systemml(signal,name):

    prog = dml(dml_script).input('signal', signal).output('DFT')
    
    return (

    #execute the script inside the SystemML engine running on top of Apache Spark
    ml.execute(prog) 
     
         #read result from SystemML execution back as SystemML Matrix
        .get('DFT') 
     
         #convert SystemML Matrix to ApacheSpark DataFrame 
        .toDF() 
     
         #rename default column names
        .selectExpr('C1 as %sa' % (name), 'C2 as %sb' % (name)) 
     
         #add unique ID per row for later joining
        .withColumn("id", monotonically_increasing_id())
    )



Now it’s time to create individual DataFrames containing only a subset of the data. We filter simultaneously for accelerometer each sensor axis and one for each class. This means you’ll get 6 DataFrames. Please implement this using the relational API of DataFrames or SparkSQL. Please use class 1 and 2 and not 0 and 1.

Please make sure that each DataFrame has only ONE colum (only the measurement, eg. not CLASS column)


In [26]:
import numpy as np
x0= spark.sql("SELECT X from df WHERE CLASS=1")
y0= spark.sql("SELECT Y from df WHERE CLASS=1")
z0= spark.sql("SELECT Z from df WHERE CLASS=1")

x1= spark.sql("SELECT X from df WHERE CLASS=2")
y1= spark.sql("SELECT Y from df WHERE CLASS=2")
z1= spark.sql("SELECT Z from df WHERE CLASS=2")



Since we’ve created this cool DFT function before, we can just call it for each of the 6 DataFrames now. And since the result of this function call is a DataFrame again we can use the pyspark best practice in simply calling methods on it sequentially. So what we are doing is the following:

    Calling DFT for each class and accelerometer sensor axis.
    Joining them together on the ID column.
    Re-adding a column containing the class index.
    Stacking both Dataframes for each classes together



In [28]:
from pyspark.sql.functions import lit
df_class_0 = dft_systemml(x0,'x') \
    .join(dft_systemml(y0,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z0,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(0))

df_class_1 = dft_systemml(x1,'x') \
    .join(dft_systemml(y1,'y'), on=['id'], how='inner') \
    .join(dft_systemml(z1,'z'), on=['id'], how='inner') \
    .withColumn('class', lit(1))
    
df_dft = df_class_0.union(df_class_1)

df_dft.show()

SystemML Statistics:
Total execution time:		0.234 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.054 sec.
Number of executed Spark inst:	0.


SystemML Statistics:
Total execution time:		0.044 sec.
Number of executed Spark inst:	0.


[Stage 18:>                                                         (0 + 8) / 8]
[Stage 18:=====================>                                    (3 + 5) / 8]
[Stage 18:===========================================>              (6 + 2) / 8]
[Stage 18:==================================================>       (7 + 1) / 8]
[Stage 20:>                                                         (0 + 8) / 8]
[Stage 20:=====================>                                    (3 + 5) / 8]
SystemML Statistics:
Total execution time:		9.750 sec.
Number of executed Spark inst:	6.

                                                                                

[Stage 26:>                                                         (

Please create a VectorAssembler which consumes the newly created DFT columns and produces a column “features”

In [29]:
from pyspark.ml.feature import VectorAssembler

In [30]:
###YOUR_CODE_GOES_HERE### 
vectorAssembler = VectorAssembler(inputCols=["xa","xb","ya","yb","za","zb"],
                                  outputCol="features") 

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to set the “class” column as target.

In [31]:
from pyspark.ml.classification import GBTClassifier

In [32]:
classifier = GBTClassifier(labelCol="class", featuresCol="features", maxIter=10)###YOUR_CODE_GOES_HERE###



Let’s train and evaluate…


In [33]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [34]:
model = pipeline.fit(df_dft)

In [35]:
prediction = model.transform(df_dft)

In [36]:
prediction.show()

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|         id|                  xa|                  xb|                  ya|                  yb|                  za|                  zb|class|            features|       rawPrediction|         probability|prediction|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+----------+
|17179869191|-0.03750866686245739|-0.00607638770223...|-0.03750866686245739|-0.00607638770223...|-0.03750866686245739|-0.00607638770223...|    0|[-0.0375086668624...|[0.83002855223973...|[0.84024566850429...|       0.0|
| 8589934592| -0.0213952544564988| -0.1156058071676539| -0.0213952544564988| -0.1156058071676539| -0.0213952544564988| -

In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("class")
    
binEval.evaluate(prediction)

0.9981761070017225

If you are happy with the result (I’m happy with > 0.8) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera.

In [38]:
!rm -Rf a2_m4.json


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [39]:
!del -Rf a2_m4.json

In [40]:
prediction = prediction.repartition(1)

In [44]:
#prediction.write.json('a2_m4.json')  # Py4JJavaError: An error occurred while calling o449.json

In [41]:
# Convert DF to PD and write json
prediction_pd = prediction.toPandas()
prediction_pd.to_csv('prediction.csv') # inspect the file

In [42]:
prediction_pd.head()

,id,xa,xb,ya,yb,za,zb,class,features,rawPrediction,probability,prediction
0,17179869191,-0.037509,-0.006076,-0.037509,-0.006076,-0.037509,-0.006076,0,"[-0.03750866686245739, -0.006076387702233552, ...","[0.8300285522397316, -0.8300285522397316]","[0.8402456685042936, 0.1597543314957064]",0.0
1,8589934592,-0.021395,-0.115606,-0.021395,-0.115606,-0.021395,-0.115606,0,"[-0.0213952544564988, -0.1156058071676539, -0....","[0.8806642236118194, -0.8806642236118194]","[0.8533759607730729, 0.14662403922692713]",0.0
2,34359738369,-0.137834,-0.063053,-0.137834,-0.063053,-0.137834,-0.063053,0,"[-0.13783372334014835, -0.06305345056979533, -...","[0.8300285522397316, -0.8300285522397316]","[0.8402456685042936, 0.1597543314957064]",0.0
3,0,-0.032326,0.085528,-0.032326,0.085528,-0.032326,0.085528,0,"[-0.032325686259876626, 0.08552771465661582, -...","[0.8300285522397316, -0.8300285522397316]","[0.8402456685042936, 0.1597543314957064]",0.0
4,25769803790,0.006261,-0.057651,0.006261,-0.057651,0.006261,-0.057651,0,"[0.006260524476137005, -0.05765058448048809, 0...","[0.8806642236118194, -0.8806642236118194]","[0.8533759607730729, 0.14662403922692713]",0.0


In [43]:
import json
prediction2Json = prediction_pd.to_json(orient="split")
predJsonParsed = json.loads(prediction2Json)
json.dumps(predJsonParsed)  

with open('a2_m4.json', 'w') as json_file:
    json.dump(predJsonParsed, json_file)

In [50]:
import os
url = 'https://raw.githubusercontent.com/IBM/coursera/master/rklib.py'
if os.path.exists("rklib.py"):
  os.remove("rklib.py")
else:
  wget.download(url)

In [54]:
#from rklib import zipit
#zipit('a2_m4.json.zip','a2_m4.json')
# it does not copy the json file to zipfolder. I use the following instead

In [55]:
import zipfile
zip_file = zipfile.ZipFile('a2_m4.json.zip', 'w')
zip_file.write('a2_m4.json', compress_type=zipfile.ZIP_DEFLATED)
zip_file.close()

In [57]:
!base64 "a2_m4.json.zip" > "a2_m4.json.zip.base64"

'base64' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from rklib import submit
key = "-fBiYHYDEeiR4QqiFhAvkA"
part = "IjtJk"
email = ###YOUR_CODE_GOES_HERE###
submission_token = ###YOUR_CODE_GOES_HERE### # (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)

with open('a2_m4.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, submission_token, key, part, [part], data)